# Final Project Submission
---

## Info

* Student name: **Barto Molina**
* Student pace: **part time**
* Scheduled project review date/time: **10/15/2019 5:00 PM (EST)**
* Instructor name: **Victor Geislinger**
* Blog post URL: [...](https://medium.com/@bartomolina/...)

## The Project

The goal of the project is to find the best 5 zipcodes across the US to invest in and model and project the median price for those zipcodes in the coming years. We're going to select these top zipcodes based on the distance to our workplace and the ROI from the past years.

We obtained the data from the [Zillow Research website](https://www.zillow.com/research/data/), which uses the ZHVI (Zillow Home Value Index) - A smoothed, seasonally adjusted measure of the median estimated home value across a given region and housing type. We've used the ZHVI for All Homes (SFR, Condo/Co-op) dataset.

## Imports

We'll import the required libraries that will be used throughout the rest of the project:

In [335]:
import pandas as pd
import numpy as np
import datetime
import googlemaps
import folium
import json
import random

In [336]:
# we use dotenv to use Google API key
# use .env.example as a reference
import os
from dotenv import load_dotenv
load_dotenv()

GOOGLE_KEY = os.getenv("GOOGLE_KEY")
gmaps = googlemaps.Client(key=GOOGLE_KEY)

## 0. Parameters
---

We will use this parameters to perform different calculations:

- arrival_time: date used for the distance calculations

In [386]:
arrival_time = datetime.datetime(2019, 11, 4, 9, 0)

## Step 1: Load the Data/Filtering for Chosen Zipcodes

In [338]:
df = pd.read_csv('zillow_data.csv', dtype={'RegionName' : str })
df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09
0,61639,10025,New York,NY,New York-Newark-Jersey City,New York County,1,167200.0,167400.0,167400.0,...,1051500,1029200,1014300,1008700,991500,973400,965800,967100,967800,965400
1,84654,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,2,148200.0,148900.0,149400.0,...,314500,314700,314600,312900,311000,309700,308900,308400,307000,305700
2,61637,10023,New York,NY,New York-Newark-Jersey City,New York County,3,350900.0,351700.0,352300.0,...,1375700,1370100,1369000,1362500,1358300,1369700,1394900,1403200,1405400,1412700
3,84616,60614,Chicago,IL,Chicago-Naperville-Elgin,Cook County,4,160800.0,162700.0,164200.0,...,367100,365900,364800,362800,360300,357600,354200,348900,343800,341500
4,61616,10002,New York,NY,New York-Newark-Jersey City,New York County,5,NaN,NaN,NaN,...,966100,954500,949600,943700,928400,913300,901400,892700,891200,889900


In [339]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4770 entries, 0 to 4769
Columns: 289 entries, RegionID to 2019-09
dtypes: float64(237), int64(47), object(5)
memory usage: 10.5+ MB


## Step 2: Data Preprocessing

In [340]:
def get_datetimes(df):
    return pd.to_datetime(df.columns.values[1:], format='%Y-%m')

In [341]:
# we're going to analyze only the NY postcodes
df_NY = df.loc[df['State'].isin(['NY', 'NJ', 'CT'])].copy()
df_NY.reset_index(drop=True, inplace=True)

In [342]:
df_NY.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09
0,61639,10025,New York,NY,New York-Newark-Jersey City,New York County,1,167200.0,167400.0,167400.0,...,1051500,1029200,1014300,1008700,991500,973400,965800,967100,967800,965400
1,61637,10023,New York,NY,New York-Newark-Jersey City,New York County,3,350900.0,351700.0,352300.0,...,1375700,1370100,1369000,1362500,1358300,1369700,1394900,1403200,1405400,1412700
2,61616,10002,New York,NY,New York-Newark-Jersey City,New York County,5,NaN,NaN,NaN,...,966100,954500,949600,943700,928400,913300,901400,892700,891200,889900
3,61807,10467,New York,NY,New York-Newark-Jersey City,Bronx County,7,NaN,NaN,NaN,...,213300,214000,215400,217600,218200,219100,223800,227600,230100,231700
4,61630,10016,New York,NY,New York-Newark-Jersey City,New York County,9,242300.0,241200.0,240000.0,...,1007700,997500,994400,993300,985100,975200,970200,964400,956700,951700


In [343]:
df_NY['CountyName'].value_counts().head()

New York County    40
Bergen County      33
Queens County      31
Kings County       31
Monmouth County    29
Name: CountyName, dtype: int64

We're going to select the zipcodes that are closer than half an hour by public transportation to the UN HQ.

In [ ]:
df_NY['Distance'] = np.nan
destination = 'United Nations Secretariat Building, East 42nd Street, New York, NY'

for i, zipcode in df_NY.iterrows():
    print (zipcode['RegionName'])
    directions_transit = gmaps.directions(zipcode['RegionName'], 'United Nations Secretariat Building, East 42nd Street, New York, NY', mode='transit', arrival_time=arrival_time)
    directions_walking = gmaps.directions(zipcode['RegionName'], 'United Nations Secretariat Building, East 42nd Street, New York, NY', mode='walking', arrival_time=arrival_time)
    if directions_transit or directions_walking:
        distance_transit = np.NaN if not directions_transit else directions_transit[0]['legs'][0]['duration']['value']
        distance_walking = np.NaN if not directions_walking else directions_walking[0]['legs'][0]['duration']['value']
        df_NY.at[i, 'Distance'] = np.nanmin([distance_walking, distance_transit]) / 60

In [402]:
df_NY.dropna(subset=['Distance'], inplace=True)
df_NY_by_dist = df_NY.loc[df_NY['Distance'] <= 50]
print(len(df_NY))
print(len(df_NY_by_dist))

466
62


In [423]:
pd.to_datetime(df_NY.columns.values[8:12], format='%Y-%m')

DatetimeIndex(['1996-05-01', '1996-06-01', '1996-07-01', '1996-08-01'], dtype='datetime64[ns]', freq=None)

In [419]:
for col in df_NY:
    print(f"{col} - {df_NY[col].isnull().sum()}")
# df_NY_by_dist.isnull().sum()

get_datetimes(df_NY)

RegionID - 0
RegionName - 0
City - 0
State - 0
Metro - 1
CountyName - 0
SizeRank - 0
1996-04 - 42
1996-05 - 42
1996-06 - 42
1996-07 - 42
1996-08 - 42
1996-09 - 42
1996-10 - 42
1996-11 - 42
1996-12 - 42
1997-01 - 42
1997-02 - 42
1997-03 - 42
1997-04 - 42
1997-05 - 42
1997-06 - 42
1997-07 - 42
1997-08 - 42
1997-09 - 42
1997-10 - 42
1997-11 - 42
1997-12 - 42
1998-01 - 42
1998-02 - 42
1998-03 - 42
1998-04 - 42
1998-05 - 42
1998-06 - 42
1998-07 - 42
1998-08 - 42
1998-09 - 42
1998-10 - 42
1998-11 - 42
1998-12 - 42
1999-01 - 42
1999-02 - 42
1999-03 - 42
1999-04 - 42
1999-05 - 42
1999-06 - 42
1999-07 - 42
1999-08 - 42
1999-09 - 42
1999-10 - 42
1999-11 - 42
1999-12 - 42
2000-01 - 42
2000-02 - 42
2000-03 - 42
2000-04 - 42
2000-05 - 42
2000-06 - 42
2000-07 - 42
2000-08 - 42
2000-09 - 42
2000-10 - 42
2000-11 - 42
2000-12 - 42
2001-01 - 42
2001-02 - 42
2001-03 - 42
2001-04 - 42
2001-05 - 42
2001-06 - 42
2001-07 - 42
2001-08 - 42
2001-09 - 42
2001-10 - 42
2001-11 - 42
2001-12 - 42
2002-01 - 42
2002-

In [405]:
ny_geo_path = 'ny_new_york_zip_codes_geo.min.json'
nj_geo_path = 'nj_new_jersey_zip_codes_geo.min.json'
ct_geo_path = 'ct_connecticut_zip_codes_geo.min.json'
with open(ny_geo_path) as ny_geo_file, open(nj_geo_path) as nj_geo_file, open(ct_geo_path) as ct_geo_file:
    NY_zip = json.load(ny_geo_file)
    NJ_zip = json.load(nj_geo_file)
    CT_zip = json.load(ct_geo_file)
    
all_features = NY_zip['features'] + NJ_zip['features'] + CT_zip['features']

tristate_data = {}
tristate_data['type'] = 'FeatureCollection'
tristate_data['features'] = list()

for zipcode in all_features:
    code = zipcode['properties']['ZCTA5CE10']
    if code in df_NY_by_dist['RegionName'].values:
        zipcode['properties']['ZCTA5CE10'] = code
        distance = df_NY_by_dist.loc[df_NY_by_dist['RegionName'] == code, 'Distance'].values[0].round(2)
        zipcode['properties']['CustLabel'] = f'{code} - {distance}m'
        tristate_data['features'].append(zipcode)

tristate_json_data = json.dumps(tristate_data)

In [406]:
m = folium.Map(location=[40.75, -73.97], zoom_start=13)

choropleth = folium.Choropleth(highlight=True,
    geo_data=tristate_json_data,
    name='choropleth',
    data=df_NY_by_dist,
    columns=['RegionName', 'Distance'],
    key_on='feature.properties.ZCTA5CE10',
    fill_color='OrRd',               
    fill_opacity=0.7,
    line_opacity=0.4,
    legend_name='house prices (median)'
).add_to(m)

folium.LayerControl().add_to(m)

# add tooltip to see the zipcode
choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['CustLabel'], labels=False)
)

m

## Step 4: Reshape from Wide to Long Format

In [ ]:
def melt_data(df):
    melted = pd.melt(df, id_vars=['RegionName', 'City', 'State', 'Metro', 'CountyName'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted.groupby('time').aggregate({'value':'mean'})

In [ ]:
df_NY.drop(['RegionID', 'SizeRank'], axis=1, inplace=True)

In [ ]:
melt_data(df_NY)

## Step 3: EDA and Visualization

In [ ]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

# NOTE: if you visualizations are too cluttered to read, try calling 'plt.gcf().autofmt_xdate()'!

## Step 5: ARIMA Modeling

## Step 6: Interpreting Results

## Future work

- Better calculate distance (currently just an average point in the zipcode)
- Better function weighting roi vs. distance